In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime

In [2]:
file_path = "Book4.xlsx"
df = pd.read_excel(file_path)

In [3]:
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['TIMESLOT'].astype(str) , format='DD/MM/YYYY')
df.asfreq('5T')

C:\Users\Sanchit\AppData\Local\Temp\ipykernel_18624\1734016011.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.asfreq('5T')


,TIMESLOT,DELHI,BRPL,BYPL,NDPL,NDMC,MES,Date
DateTime,,,,,,,,
2021-01-01 00:00:00,00:00:00,2318.54,973.49,452.98,711.43,129.75,28.74,01/01/2021
2021-01-01 00:05:00,00:05:00,2238.05,933.48,420.41,702.30,131.75,28.40,01/01/2021
2021-01-01 00:10:00,00:10:00,2192.71,925.58,404.30,680.26,134.17,27.69,01/01/2021
2021-01-01 00:15:00,00:15:00,2158.28,908.45,404.75,662.20,133.95,27.36,01/01/2021
2021-01-01 00:20:00,00:20:00,2046.05,793.02,409.20,664.50,131.94,27.41,01/01/2021
...,...,...,...,...,...,...,...,...
2024-12-11 22:20:00,22:20:00,3159.38,1388.72,667.26,941.52,117.81,15.43,12/11/2024
2024-12-11 22:25:00,22:25:00,3152.87,1379.12,682.69,927.29,120.08,15.36,12/11/2024
2024-12-11 22:30:00,22:30:00,3119.45,1371.69,652.06,918.50,130.66,15.20,12/11/2024


In [4]:
df.drop(columns=['Date', 'TIMESLOT'], inplace=True, errors='ignore')
df.sort_index(inplace=True)

In [5]:
start_test_date = "2024-06-07"
end_test_date = "2024-11-12"

train_df = df[df.index < start_test_date]
test_df = df[(df.index >= start_test_date) & (df.index <= end_test_date)]

In [6]:
def fit_and_forecast_sarima(train_series, test_series, order=(1,1,1), seasonal_order=(0,1,5,12)):
    model = SARIMAX(train_series,
                    order=order,
                    seasonal_order=seasonal_order,
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    # Forecast the length of the test period
    forecast_steps = len(test_series)
    forecast = model_fit.forecast(steps=forecast_steps)
    forecast.index = test_series.index  # Align forecast index with test_series
    return forecast


In [7]:
columns_to_model = ['DELHI', 'BRPL', 'BYPL', 'NDPL', 'NDMC', 'MES']
results = {}

In [8]:
df = df.resample('5T').mean()

C:\Users\Sanchit\AppData\Local\Temp\ipykernel_18624\1224181088.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('5T').mean()


In [9]:
for col in columns_to_model:
    train_series = train_df[col].dropna()
    test_series = test_df[col].dropna()

    if train_series.empty or test_series.empty:
        print(f"Skipping {col} due to lack of data in train or test.")
        continue

    # Fit and forecast SARIMA model
    # Adjust orders after exploration (p,d,q,P,D,Q,m)
    forecast = fit_and_forecast_sarima(train_series, test_series, order=(1,1,1), seasonal_order=(0,1,5,12))
    print("done with one column")
    # Store results
    results[col] = {
        'actual': test_series,
        'forecast': forecast
    }

C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodel

done with one column


C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Sanchit\AppData\Local\pypoetry\Cache\virtualenvs\sih2-Xq_45JKi-py3.13\Lib\site-package

MemoryError: Unable to allocate 1.84 GiB for an array with shape (75, 75, 43950) and data type float64

In [ ]:
def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

print("Accuracy Metrics:")
for col, res in results.items():
    actual = res['actual']
    forecast = res['forecast']
    mae = mean_absolute_error(actual, forecast)
    mape = mean_absolute_percentage_error(actual, forecast)
    rmse = root_mean_squared_error(actual, forecast)

    print(f"--- {col} ---")
    print(f"MAE:  {mae:.3f}")
    print(f"MAPE: {mape:.3f}%")
    print(f"RMSE: {rmse:.3f}\n")
